# **Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from keras import models, layers, metrics
from keras.callbacks import EarlyStopping
from keras.models import load_model, Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, MaxPooling2D, MaxPool2D
from keras.preprocessing import image
from keras import losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# **One time execution**

In [ ]:
def categorise_five_classes(row):
    if row['Wind Speed'] <= 33:
        return 'T. Depression'
    elif row['Wind Speed'] <= 64:
        return 'T. Storm'
    elif row['Wind Speed'] <= 83:
        return 'Category 1'
    elif row['Wind Speed'] <= 95:
        return 'Category 2'
    return 'Category 3'

In [ ]:
def categorise_seven_classes(row):
    if row['Wind Speed'] <= 33:
        return 'T. Depression'
    elif row['Wind Speed'] <= 64:
        return 'T. Storm'
    elif row['Wind Speed'] <= 83:
        return 'Category 1'
    elif row['Wind Speed'] <= 95:
        return 'Category 2'
    elif row['Wind Speed'] <= 113:
        return 'Category 3'
    elif row['Wind Speed'] <= 134:
        return 'Category 4'
    return 'Category 5'

In [ ]:
label_path_NA = '/content/drive/MyDrive/HURSAT/hursat/NA_labels.npy'
label_path_NP = '/content/drive/MyDrive/HURSAT/hursat/NP_labels.npy'
label_path_NI = '/content/drive/MyDrive/HURSAT/hursat/NI_labels.npy'
label_path_NP_NA = '/content/drive/MyDrive/HURSAT/hursat/NP_NA_labels.npy'
label_path_NP_NI_NA = '/content/drive/MyDrive/HURSAT/hursat/NP_NI_NA_labels.npy'

In [ ]:
data = np.load(label_path_NA)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_five_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NA_five_class_labels.csv')

In [ ]:
data = np.load(label_path_NA)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_seven_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NA_seven_class_labels.csv')

In [ ]:
data = np.load(label_path_NP)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_five_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NP_five_class_labels.csv')

In [ ]:
data = np.load(label_path_NP)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_seven_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NA_seven_class_labels.csv')

In [ ]:
data = np.load(label_path_NP_NA)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_five_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NP_NA_five_class_labels.csv')

In [ ]:
data = np.load(label_path_NP_NA)
df = pd.DataFrame(data, columns = ['Wind Speed'])
df['Wind Speed'] = df.apply(lambda row: categorise_seven_classes(row), axis = 1)
df.to_csv('/content/drive/MyDrive/HURSAT/hursat/NP_NA_seven_class_labels.csv')

# **Classification**

*Training: NA images, Testing: NI images, No. of classes: 5, Model: LeNet*

In [ ]:
num_classes = 5
class_names = ['Tropical Depression',"Tropical Storm", 'Category 1', 'Category 2', 'Category 3']

train_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NA_images.npy')
train_data = np.reshape(train_data, (14296, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data/255
test_data = test_data/255

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/HURSAT/hursat/NA_five_class_labels.csv')
train_label_data = pd.Series(train_labels['Wind Speed'])

In [ ]:
lb = LabelBinarizer()
train_label_data = lb.fit_transform(train_label_data)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label_data, test_size = 0.2, random_state = 43)

In [ ]:
save_at = "/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NA_five_classes.hdf5"
save_best = ModelCheckpoint(save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
# Load the Proposed Model
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3, 3), input_shape = (50, 50, 1), activation = 'relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
tf.keras.layers.Dropout(0.4),

tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
tf.keras.layers.Dropout(0.4),

tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
tf.keras.layers.Dropout(0.4),

tf.keras.layers.Flatten(),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(units = 512, activation = 'relu'),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32,(5,5), activation="relu", input_shape=(50,50,1), padding='same', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size = 4, epochs = 40, verbose = 1, validation_data = (X_val, Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'], 'g')
plt.plot(history.history['val_accuracy'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

plt.figure(figsize = (7, 7))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

In [ ]:
model = load_model('/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NA_five_classes.hdf5')

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
pred = model.predict(test_data, verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_labels.npy')
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    else:
        temp.append(4)

In [ ]:
test_accuracy = np.mean(pred_labels == temp)
print("Test accuracy:", test_accuracy)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names)
fig, ax = plt.subplots(figsize = (7, 7))
disp.plot(ax = ax);

# **Classification**

*Training: NP images, Testing: NI images, No. of classes: 5, Model: LeNet*

In [ ]:
num_classes = 5
class_names = ['Tropical Depression', "Tropical Storm", 'Category 1', 'Category 2', 'Category 3']

train_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NP_images.npy')
train_data = np.reshape(train_data, (6189, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data/255
test_data = test_data/255

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/HURSAT/hursat/NP_five_class_labels.csv')
train_label_data = pd.Series(train_labels['Wind Speed'])

In [ ]:
lb = LabelBinarizer()
train_label_data = lb.fit_transform(train_label_data)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label_data, test_size = 0.2, random_state = 431)

In [ ]:
save_at = "/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NP_five_classes.hdf5"
save_best = ModelCheckpoint (save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32,(5,5), activation="relu", input_shape=(50,50,1), padding='same', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size = 4, epochs = 40, verbose = 1, validation_data = (X_val, Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'], 'g')
plt.plot(history.history['val_accuracy'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

plt.figure(figsize = (7, 7))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

In [ ]:
model = load_model('/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NP_five_classes.hdf5')

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
pred = model.predict(test_data, verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_labels.npy')
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    else:
        temp.append(4)

In [ ]:
test_accuracy = np.mean(pred_labels == temp)
print("Test accuracy:", test_accuracy)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names)
fig, ax = plt.subplots(figsize = (7, 7))
disp.plot(ax = ax);

# **Classification**

*Training: NP + NA images, Testing: NI images, No. of classes: 5, Model: LeNet*

In [ ]:
num_classes = 5
class_names = ['Tropical Depression',"Tropical Storm", 'Category 1', 'Category 2', 'Category 3']

train_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NP_NA_images.npy')
train_data = np.reshape(train_data, (20485, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data/255
test_data = test_data/255

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/HURSAT/hursat/NP_NA_five_class_labels.csv')
train_label_data = pd.Series(train_labels['Wind Speed'])

In [ ]:
lb = LabelBinarizer()
train_label_data = lb.fit_transform(train_label_data)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label_data, test_size = 0.2, random_state = 431)

In [ ]:
save_at = "/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NP_NA_five_classes.hdf5"
save_best = ModelCheckpoint (save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32,(5,5), activation="relu", input_shape=(50,50,1), padding='same', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation="relu", padding='valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size = 4, epochs = 40, verbose = 1, validation_data = (X_val, Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'], 'g')
plt.plot(history.history['val_accuracy'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

plt.figure(figsize = (7, 7))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
pred = model.predict(test_data, verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_labels.npy')
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    else:
        temp.append(4)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names)
fig, ax = plt.subplots(figsize = (7, 7))
disp.plot(ax = ax);

# **Classification**
*Training: NP images, Testing: NI images, No. of classes: 7, Model: LeNet*

In [ ]:
num_classes = 7
class_names = ['Tropical Depression', "Tropical Storm", 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']

train_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NP_images.npy')
train_data = np.reshape(train_data, (6189, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data / 255
test_data = test_data / 255

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/HURSAT/hursat/NP_seven_class_labels.csv')
train_label_data = pd.Series(train_labels['Wind Speed'])

In [ ]:
lb = LabelBinarizer()
train_label_data = lb.fit_transform(train_label_data)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label_data, test_size = 0.2, random_state = 431)

In [ ]:
save_at = "/content/drive/MyDrive/hursat/lenet5_classification_model_NP_seven_classes.hdf5"
save_best = ModelCheckpoint (save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation = "relu", input_shape = (50, 50, 1), padding = 'same', strides = 1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = "relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate, beta_1 = 0.9, beta_2 = 0.999), metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size = 8, epochs = 40, verbose = 1, validation_data = (X_val,Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'], 'g')
plt.plot(history.history['val_accuracy'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

plt.figure(figsize = (7, 7))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')
plt.xticks(np.arange(1, 40, 2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
pred = model.predict(test_data, verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/hursat/NI_labels.npy')
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    elif i <= 113:
        temp.append(4)
    elif i <= 134:
        temp.append(5)
    else:
        temp.append(6)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names)
fig, ax = plt.subplots(figsize = (7, 7))
disp.plot(ax = ax);

# **Classification**
*Training: NA images, Testing: NI images, No. of classes: 7, Model: LeNet*

In [ ]:
num_classes = 7
class_names = ['Tropical Depression',"Tropical Storm", 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']

train_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NA_images.npy')
train_data = np.reshape(train_data, (14296, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/HURSAT/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data/255
test_data = test_data/255

In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/HURSAT/hursat/NA_seven_class_labels.csv')

In [ ]:
label_data = pd.Series(labels['Wind Speed'])

In [ ]:
#Normalization
lb = LabelBinarizer()
label_data = lb.fit_transform(label_data)

In [ ]:
X_train,X_val,Y_train,Y_val = train_test_split(train_data, label_data, test_size = 0.2, random_state = 43)

In [ ]:
save_at = "/content/drive/MyDrive/HURSAT/hursat/lenet5_classification_model_NA_seven_classes.hdf5"
save_best = ModelCheckpoint (save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation = "relu", input_shape = (50, 50, 1), padding = 'same', strides = 1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = "relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
#Compile the model
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate, beta_1 = 0.9, beta_2 = 0.999), metrics = ['accuracy'])

In [ ]:
#Training the model
history = model.fit(X_train, Y_train, batch_size = 16, epochs = 40, verbose = 1, validation_data = (X_val, Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
#Plots for training and validation process:loss and accuracy
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'],'g')
plt.plot(history.history['val_accuracy'],'b')
plt.xticks(np.arange(1,40,2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train','validation'])
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

plt.figure(figsize=(7,7))
plt.plot(history.history['loss'],'g')
plt.plot(history.history['val_loss'],'b')
plt.xticks(np.arange(1,40,2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train','validation'])
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
#predict the model
pred = model.predict(test_data,verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/hursat/NI_labels.npy')
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    elif i <= 113:
        temp.append(4)
    elif i <= 134:
        temp.append(5)
    else:
        temp.append(6)

len(temp)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize = (10,10))
disp.plot(ax = ax);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(temp, pred_labels, target_names = class_names))

In [ ]:
index = np.argmax(pred,axis=1)
labels = lb.classes_[index]
labels

In [ ]:
def classification_evaluation(classification, predicted_labels, test_labels, test_images):
    if classification== "correct":
        indices_list = np.where(predicted_labels == test_labels)[0]
    else:
        indices_list = np.where(predicted_labels!= test_labels)[0]
    test_images_filtered= [test_images[i] for i in indices_list]
    images_labels_original= [test_labels[i] for i in indices_list]
    images_labels_predicted= [pred_labels[i] for i in indices_list]
    print(f"{len(test_images_filtered)} images were classified {classification.upper()} out of a total of {len(test_images)} in the Test dataset")

    unique, counts = np.unique(images_labels_original, return_counts=True)
    for i in range(0,len(unique)):
        print(f"for category {unique[i]} the number of {classification.upper()} classified images were: {counts[i]}")


    '''
    # Plot some of the misclassified images
    print("\n\n")
    fig,ax=plt.subplots(5,2)
    fig.suptitle(f"Sample of {classification.upper()} Classified Images", fontsize=20)
    fig.set_size_inches(15,15)
    for i in range(5):
        for j in range (2):
            l=random.randint(0,len(test_images_filtered))
            ax[i,j].imshow(test_images_filtered[l])
            ax[i,j].set_title("Predicted: "+str(images_labels_predicted[l])+"\n"+"Actual: "+str(images_labels_original[l]))
    plt.tight_layout()
    '''

In [ ]:
classification_evaluation("incorrect", pred_labels, temp, test_data)

In [ ]:
classification_evaluation("correct", pred_labels, temp, test_data)

# **Classification**
*Training: NP + NA, Testing: NI images, No. of classes: 7, Model: LeNet*

In [ ]:
num_classes = 7
class_names = ['Tropical Depression',"Tropical Storm", 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']

train_data = np.load('/content/drive/MyDrive/hursat/NP_NA_images.npy')
train_data = np.reshape(train_data, (20485, 50, 50, 1))

test_data = np.load('/content/drive/MyDrive/hursat/NI_images.npy')
test_data = np.reshape(test_data,(195, 50, 50, 1))

train_data = train_data / 255
test_data = test_data / 255

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/hursat/NP_NA_seven_class_labels.csv')
train_label_data = pd.Series(train_labels['Wind Speed'])

In [ ]:
lb = LabelBinarizer()
train_label_data = lb.fit_transform(train_label_data)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label_data, test_size = 0.2, random_state = 431)

In [ ]:
save_at = "/content/drive/MyDrive/hursat/lenet5_classification_model_NP_NA_seven_classes.hdf5"
save_best = ModelCheckpoint (save_at, monitor = 'val_accuracy', verbose = 0, save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
#Lenet Model
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation = "relu", input_shape = (50, 50, 1), padding = 'same', strides = 1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation = "relu", padding = 'valid', strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = "relu"))
model.add(Dropout(0.3))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.compile(loss = losses.categorical_crossentropy, optimizer = Adam(learning_rate, beta_1 = 0.9, beta_2 = 0.999), metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size = 16, epochs = 50, verbose = 1, validation_data = (X_val, Y_val), shuffle = True, callbacks = [save_best])

In [ ]:
plt.figure(figsize = (7, 7))
plt.plot(history.history['accuracy'], 'g')
plt.plot(history.history['val_accuracy'], 'b')
plt.xticks(np.arange(1, 50, 2))
plt.title('training accuracy vs validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

plt.figure(figsize = (7, 7))
plt.plot(history.history['loss'], 'g')
plt.plot(history.history['val_loss'], 'b')
plt.xticks(np.arange(1, 50, 2))
plt.title('training loss vs validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('loss')
plt.legend(['train', 'validation'])
plt.grid(color = 'y', linewidth='0.5')
plt.show()

In [ ]:
score, acc = model.evaluate(X_val, Y_val)
print('Validation Accuracy:', acc)

In [ ]:
pred = model.predict(test_data, verbose = 1)
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
test_labels = np.load('/content/drive/MyDrive/hursat/NI_labels.npy')

In [ ]:
temp = []
for i in test_labels:
    if i <= 33:
        temp.append(0)
    elif i <= 64:
        temp.append(1)
    elif i <= 83:
        temp.append(2)
    elif i <= 95:
        temp.append(3)
    elif i <= 113:
        temp.append(4)
    elif i <= 134:
        temp.append(5)
    else:
        temp.append(6)

In [ ]:
cm = confusion_matrix(temp, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names)
fig, ax = plt.subplots(figsize = (10, 10))
disp.plot(ax = ax);